In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from tensorflow.keras.layers import BatchNormalization

#from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping


import matplotlib.pyplot as plt


In [2]:
# Import our dataset 
#df = pd.read_csv('/kaggle/input/religious-tweets/tweetsDF.csv', encoding='utf-16')
df = pd.read_csv('/kaggle/input/test-tweets/tweets_test.csv', encoding='utf-16')
df.head()


,id,tweet
0,958390144494694400,اخطر اعداء الامه منافقوها ينشغلون بتشويه مصلحي...
1,958390531867971584,حتي مارك اليهودي يحب مخرجات مءتمر الخوار الوطن...
2,958390679780016129,gcc تصوير جوي لحصن مرحب في خيبر الذي فتحه النب...
3,958390811514822656,خدمات الكنيسه ماشيين مع بعض بمبدا اليهود لا يع...
4,958390828682096640,ذكر الله المنافقين اكثر من اليهود لانهم يستعمل...


In [5]:
train_df = pd.read_csv('/kaggle/working/test_tweets.csv', encoding='utf-16')
train_df.head()

,id,tweet,hate
0,958390144494694400,اخطر اعداء الامه منافقوها ينشغلون بتشويه مصلحي...,1
1,958390531867971584,حتي مارك اليهودي يحب مخرجات مءتمر الخوار الوطن...,0
2,958390679780016129,gcc تصوير جوي لحصن مرحب في خيبر الذي فتحه النب...,0
3,958390811514822656,خدمات الكنيسه ماشيين مع بعض بمبدا اليهود لا يع...,0
4,958390828682096640,ذكر الله المنافقين اكثر من اليهود لانهم يستعمل...,0


In [27]:
# Getting max number of words coz this will help us in padding later 
train['tweet'].apply(lambda x: len(str(x).split())).max()

65

In [28]:
# Writing a function to getting AUC score for validation: 
# Understand ROC vs AUC score 
def auc(pred, target):
    fpr, tpr, threshold = metrics.roc_curve(target, pred)
    return (metrics.auc(fpr, tpr))

In [29]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.tweet.values, train.hate.values, test_size = 0.2, random_state = 42, stratify= train.hate.values, shuffle = True)

## Simple RNN:

In [30]:
# Using Keras Tokenizer:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
token = Tokenizer()
max_len = 65
token.fit_on_texts(list(xtrain) + list(xvalid))

In [31]:
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)
xtrain_pad = pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = pad_sequences(xvalid_seq, maxlen=max_len)

In [32]:
word_index = token.word_index

In [33]:
# RNN model 
model = Sequential()
model.add(Embedding(len(word_index)+1, 300, input_length=65))
model.add(SimpleRNN(100))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss= 'binary_crossentropy', optimizer= 'adam', metrics=['accuracy'])
model.summary()

2023-01-03 16:35:10.297559: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 65, 300)           5538000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 100)               40100     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 5,578,201
Trainable params: 5,578,201
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.fit(xtrain_pad, ytrain, epochs = 5)

2023-01-03 16:35:17.328171: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5
80/80 [==============================] - 9s 93ms/step - loss: 0.6364 - accuracy: 0.6171
Epoch 2/5
80/80 [==============================] - 7s 93ms/step - loss: 0.2071 - accuracy: 0.9435
Epoch 3/5
80/80 [==============================] - 8s 100ms/step - loss: 0.1147 - accuracy: 0.9605
Epoch 4/5
80/80 [==============================] - 7s 94ms/step - loss: 0.0892 - accuracy: 0.9637
Epoch 5/5
80/80 [==============================] - 8s 94ms/step - loss: 0.0856 - accuracy: 0.9652


In [35]:
scores = model.predict(xvalid_pad)
auc(scores, yvalid)

0.7176904379351543